In [30]:
import re, string

In [64]:
import pandas as pd

# read validation data file and write to another file.
with open('./ner_tagged.tsv', 'r') as outf, open("cleaned_ner_tagged.tsv", 'w') as inf:
    for line in outf:
        line = line.strip()
        if line == "":
            inf.write("\t\t\t\n")
            continue
        word, x, _, tag = line.split("\t")
        if word.replace(".", "", 1).isdigit():
          # if . appears in a digit.
            inf.write(word+"\t"+x+"\t"+_+"\t"+tag+"\n")
            continue
        parts = []
        for match in re.finditer(r'[^.,?!\s]+|[.,?!]', word):
             parts.append(match.group())
        for p in parts:
            if p in string.punctuation:
            # if p is a punctuation.
                tag = "O"
                inf.write(p+"\t"+x+"\t"+_+"\t"+tag+"\n")
            else:
            # if p is a word, then correct the spell and write to new file.
                p = spell(p)
                inf.write(p+"\t"+x+"\t"+_+"\t"+tag+"\n")



In [65]:
data = pd.read_csv('./cleaned_ner_tagged.tsv', sep='\t')
data.rename(columns = {'O':'Tags','-DOCSTART-':'Tokens','-X-':'X'}, inplace = True)

In [66]:
data

,Tokens,X,_,Tags
0,I,-X-,_,O
1,am,-X-,_,O
2,looking,-X-,_,O
3,for,-X-,_,O
4,a,-X-,_,O
...,...,...,...,...
2670,30,-X-,_,B-SIZE
2671,inch,-X-,_,I-SIZE
2672,white,-X-,_,B-COLOUR
2673,desk,-X-,_,B-PRODUCT


In [67]:
!python3 -m pip install --user pyspellchecker
from spellchecker import SpellChecker

spell2 = SpellChecker()

In [68]:
import csv
#Create attributes file from https://nijianmo.github.io/amazon/index.html
known_attributes = set()
with open('attributes.csv', newline='\n') as csvfile:
    attributes = csv.reader(csvfile, delimiter=' ')
    for row in attributes:
        for attr in row:
            known_attributes.add(attr.lower())

In [70]:
neg_sents = []
current_sent = []
current_tags = []
contains_neg = False
non_neg = 0
all_sents = []

for i, row in data.reset_index().iterrows():
    if type(row.Tags) == str:
        corrected = row.Tokens
        if row.Tokens.isalpha() and row.Tokens.lower() not in known_attributes:
            corrected = spell2.correction(row.Tokens)#spell(row.Tokens)
#         if corrected != row.Tokens:
#             print(f"{row.Tokens} changed to {corrected}")
        current_sent.append((corrected, row.Tags))
        if "-N-" in row.Tags:
            contains_neg = True
    else:
        all_sents.append(current_sent)
        current_sent = []
        contains_neg = False

all_sents

[[('I', 'O'),
  ('am', 'O'),
  ('looking', 'O'),
  ('for', 'O'),
  ('a', 'O'),
  ('black', 'B-COLOUR'),
  ('gloss', 'B-TEXTURE'),
  ('33', 'B-SIZE'),
  ('inch', 'I-SIZE'),
  ('foreplay', 'B-PRODUCT'),
  ('apron', 'I-PRODUCT'),
  ('sink', 'I-PRODUCT')],
 [('Looking', 'O'),
  ('for', 'O'),
  ('pre', 'B-PRODUCT'),
  ('workout', 'I-PRODUCT'),
  ('Pump', 'I-PRODUCT'),
  ('addict', 'I-PRODUCT'),
  ('instead', 'O'),
  ('of', 'O'),
  ('carolyn', 'B-N-PRODUCT'),
  ('Hydrate', 'I-N-PRODUCT'),
  ('With', 'O'),
  ('one', 'O'),
  ('is', 'O'),
  ('better', 'O'),
  ('?', 'O')],
 [('i', 'O'),
  ('need', 'O'),
  ('a', 'O'),
  ('48', 'B-SIZE'),
  ('inch', 'I-SIZE'),
  ('glass', 'B-MATERIAL'),
  ('sliding', 'B-PRODUCT'),
  ('good', 'I-PRODUCT'),
  ('and', 'O'),
  ('a', 'O'),
  ('shower', 'B-PRODUCT'),
  ('pan', 'I-PRODUCT'),
  ('system', 'I-PRODUCT'),
  ('for', 'O'),
  ('500.99', 'B-PRICE'),
  ('or', 'I-PRICE'),
  ('less', 'I-PRICE')],
 [('Hello', 'O'),
  (',', 'O'),
  ('do', 'O'),
  ('any', 'O'),
  ('of

In [72]:
for i in range(len(all_sents)):
    print(" ".join(word for word, tag in all_sents[i]))
    print()
    print(" ".join(word + "-" + tag for word, tag in all_sents[i]))


I am looking for a black gloss 33 inch foreplay apron sink

I-O am-O looking-O for-O a-O black-B-COLOUR gloss-B-TEXTURE 33-B-SIZE inch-I-SIZE foreplay-B-PRODUCT apron-I-PRODUCT sink-I-PRODUCT
Looking for pre workout Pump addict instead of carolyn Hydrate With one is better ?

Looking-O for-O pre-B-PRODUCT workout-I-PRODUCT Pump-I-PRODUCT addict-I-PRODUCT instead-O of-O carolyn-B-N-PRODUCT Hydrate-I-N-PRODUCT With-O one-O is-O better-O ?-O
i need a 48 inch glass sliding good and a shower pan system for 500.99 or less

i-O need-O a-O 48-B-SIZE inch-I-SIZE glass-B-MATERIAL sliding-B-PRODUCT good-I-PRODUCT and-O a-O shower-B-PRODUCT pan-I-PRODUCT system-I-PRODUCT for-O 500.99-B-PRICE or-I-PRICE less-I-PRICE
Hello , do any of your free standing stubs have a matte white finish instead of a glossy finish ?

Hello-O ,-O do-O any-O of-O your-O free-B-ATTRIBUTE standing-I-ATTRIBUTE stubs-B-PRODUCT have-O a-O matte-B-TEXTURE white-B-COLOUR finish-O instead-O of-O a-O glossy-B-N-TEXTURE finish-O ?